In [ ]:
import re
from glob import glob

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
files = glob('../../../data/collapsedcone/kernels/original/scaf*')

def extract_energy_from_filename(filename):
    return int(re.search('scaf(\d+)$', filename).group(1)) / 1000

energy_levels = np.sort([
    extract_energy_from_filename(filename)
    for filename in files
])

In [ ]:
def voxel_volume(r1, r2, theta1, theta2):
    volume = 2/3 * np.pi * (r2**3 - r1**3) * (np.cos(theta1) - np.cos(theta2))
    return volume

In [ ]:
radii_filepath = '../../../data/collapsedcone/kernels/original/voxels.dat'
loaded_radii_boundaries = pd.read_csv(radii_filepath, header=None)[0].values

radii_boundaries = np.concatenate([[0], loaded_radii_boundaries])
radii = radii_boundaries[0:-1] + (radii_boundaries[1::] - radii_boundaries[0:-1]) / 2

In [ ]:
dtheta = np.pi / 48
angle_boundaries = np.arange(0, np.pi + dtheta, dtheta)

radians = angle_boundaries[0:-1] + (angle_boundaries[1::] - angle_boundaries[0:-1]) / 2
degrees = radians * 180 / np.pi

In [ ]:
theta_mesh, radii_mesh = np.meshgrid(degrees/180 *np.pi, radii)

xx = radii_mesh * np.cos(theta_mesh)
yy = radii_mesh * np.sin(theta_mesh)

In [ ]:
voxel_volumes_mesh = np.array([
    [
        voxel_volume(
            radii_boundaries[i], radii_boundaries[i+1], 
            angle_boundaries[j], angle_boundaries[j+1])
        for j in range(len(angle_boundaries) - 1)
    ] for i in range(len(radii_boundaries) - 1)
])
voxel_mass_mesh = voxel_volumes_mesh / 1000  # cm^3 of water --> kg

In [ ]:
index_map = {
    0: 'primary',
    1: 'first_scatter',
    2: 'second_scatter',
    3: 'multiple_scatter',
    4: 'bremsstrahlung_and_annihilation',
    5: 'mean_radius',
    6: 'mean_angle'
}

In [ ]:
mev_to_joule = 1.6021773e-13

In [ ]:
def get_item_from_kernel_file(kernel_file, index):
    column_1 = kernel_file.iloc[index::7, 0].values
    
    return np.reshape(column_1, (len(degrees), len(radii))).T


def get_kernel_data(kernel_energy): # energy in MeV
    kernel_filepath = '../../../data/collapsedcone/kernels/original/scaf{}'.format(
        int(kernel_energy * 1000))
    kernel_file = pd.read_csv(kernel_filepath, header=None, delimiter='\s+')
    
    dose_index = range(5)
    
    data = {
        index_map[index]: (
            get_item_from_kernel_file(kernel_file, index)
            * mev_to_joule / voxel_mass_mesh * 100  # cGy / MeV / photon
        )
        for index in dose_index
    }
    
    data['total_dose'] = (
        data['primary'] +
        data['first_scatter'] + 
        data['second_scatter'] +
        data['multiple_scatter'] +
        data['bremsstrahlung_and_annihilation']
    )
    
    return data

In [ ]:
paper_data = {
    kernel_energy: get_kernel_data(kernel_energy)
    for kernel_energy in energy_levels
}

In [ ]:
plt.figure(figsize=(12,5))
plt.contour(xx, yy, np.log10(paper_data[1.25]['primary']), levels=[-11, -10, -9, -8])
plt.colorbar()
plt.axis('equal');
plt.xlim([-0.5,0.5])
plt.ylim([0,0.5])
plt.title('Replicating Figure 4');

In [ ]:
plt.figure(figsize=(12,5))
plt.contour(xx, yy, np.log10(paper_data[1.25]['first_scatter']), levels=[-13, -12, -11, -10])
plt.colorbar()
plt.axis('equal');
plt.xlim([-10,10])
plt.ylim([0,10])
plt.title('Replicating Figure 5');

In [ ]:
plt.figure(figsize=(12,5))
plt.contour(xx, yy, np.log10(paper_data[1.25]['total_dose']), levels=[-13, -12, -11, -10])
plt.colorbar()
plt.axis('equal');
plt.xlim([-10,10])
plt.ylim([0,10])
plt.title('Replicating Figure 6');

In [ ]:
plt.figure(figsize=(12,5))
plt.contour(xx, yy, np.log10(paper_data[0.1]['total_dose']), levels=[-12, -11, -10])
plt.colorbar()
plt.axis('equal');
plt.xlim([-10,10])
plt.ylim([0,10])
plt.title('Replicating Figure 7');

In [ ]:
plt.figure(figsize=(12,5))
plt.contour(xx, yy, np.log10(paper_data[10]['total_dose']), levels=[-13, -12, -11, -10])
plt.colorbar()
plt.axis('equal');
plt.xlim([-10,10])
plt.ylim([0,10])
plt.title('Replicating Figure 8');